# Applio UI
An RVC Fork, for Voice Conversion, with extra features like TTS.

[IA Hispano Discord Support](https://discord.gg/urxFjYmYYh) — [AI HUB Discord Support](https://discord.gg/aihub) — [GitHub](https://github.com/IAHispano/Applio) — [Terms of Use](https://github.com/IAHispano/Applio/blob/main/TERMS_OF_USE.md)


## Credits
- Original/Mainline RVC: [RVC Project](https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI)
- Applio and original Cloud Ports: [IA Hispano](https://github.com/IAHispano)
- Lightning.AI Port (with other tunnels): [Nick088](https://linktr.ee/Nick088)

#### **Disclaimer**
By using Applio and this Lightning.AI Cloud Port, you agree to comply with ethical and legal standards, respect intellectual property and privacy rights, avoid harmful or prohibited uses, and accept full responsibility for any outcomes, while Applio & Nick088 disclaims liability and reserves the right to amend these terms.

# Installation
It may take a few minutes.

It will be persistent since it's Lightning.AI, and won't need to re-run it unless you need to update, by deleting everything and re-running the whole notebook.

## (Optional) Branch
You can choose which branch of Applio to use, such as the "main" branch for the latest not stable yet updates, or a stable release version like "3.2.9".

Default Value: "3.2.9"

In [ ]:
branch = "3.2.9"


# DON'T TOUCH ANYTHING DOWN BELOW!


from IPython.display import clear_output

url = "https://github.com/IAHispano/Applio"

%cd /teamspace/studios/this_studio

!git clone --depth 1 $url --branch $branch --single-branch

print("Installing requirements...")
%cd /teamspace/studios/this_studio/Applio
!pip install uv
!uv pip install -r requirements.txt

# to kill previous instances
!sudo apt-get update
!sudo apt-get install psmisc

print("Downloading pretrained models...")
!python core.py "prerequisites" --models "True" --pretraineds_hifigan "True"

clear_output()
print("Installed!")

# Run UI

This cell will start Applio via Tunnels, the first time that you run it will download the models, so it can take a few minutes (usually ~1-2 minutes)

## Tunnels

Select he type of tunnel you wanna use for seeing the public link, so that if one of them is down, you can use the other one:

 A) Gradio: Select it in Tunnel, Run the cell and click the Gradio Public Link Url.

 B) Port Viewer: Select it in Tunnel, Click the + at the bottom of the right tab, click on Web Apps and install Port Viewer, run the cell, then wait for the Local URL to appear and click on Port Viewer on the right tab, Add a new port and write 6969 as the Port Number and optionally give it a name, after it's added, double click on the port name to see the UI, and to see it bigger you can click Open.

 C) Ngrok: Select it in Tunnel, get the Ngrok Tunnel Authtoken here: https://dashboard.ngrok.com/tunnels/authtokens/new, run the cell, wait for the Local URL to appear and click on the Ngrok Tunnel Public URL which is above. It has a 1GB Bandwidth Free Monthly Limit https://ngrok.com/docs/pricing-limits/free-plan-limits/.

 D) Cloudflare: Select it in Tunnel, run the cell, wait for the Local URL to appear and click on the Cloudflare Tunnel Public URL which is above.

 E) LocalTunnel: Select it in Tunnel, run the cell, wait for the Local URL to appear, copy the LocalTunnel Password displayed under the public link and paste it in Tunnel Password of the LocalTunnel Tunnel Public URL which is above.

 F) Horizon: Select it in Tunnel, get the Horizon ID here: https://hrzn.run/dashboard/ , login, on the 2nd step, it shows an hrzn login YOURID, you need to copy that id and put it in horizonid. Then run the cell, you will get an 'HORIZON: Authorize at https://hrzn.run/dashboard/settings/cli-token-requests/YOUR_ID', click it, do Approve. At the end, run the cell, wait for the Local URL to appear and click on the Horizon Tunnel Public URL which is above.

## (Optional) Ngrok Region

If you're using the Ngrok Tunnel, you can change to a region near to you or keep at United States if increase latency.

Default Value: ap - Asia/Pacific (Singapore)


## (Optional) Tensorboard

The Tensorboard is used for Training only, to monitor how it's going. Start it via the Lightning.AI right-hand sidebar.

In [ ]:
Tunnel = "Port Viewer" #@param ["Gradio", "Port Viewer", "Ngrok", "Cloudflare", "LocalTunnel", "Horizon"]

Token = "Ngrok | Horizon TOKEN" # @param {type:"string"}
Region = "ap - Asia/Pacific (Singapore)" # @param ["ap - Asia/Pacific (Singapore)", "au - Australia (Sydney)","eu - Europe (Frankfurt)", "in - India (Mumbai)","jp - Japan (Tokyo)","sa - South America (Sao Paulo)", "us - United States (Ohio)"]


# DON'T TOUCH ANYTHING DOWN BELOW!


PORT = 6969

import json
import time
from IPython.display import clear_output

%cd /teamspace/studios/this_studio/


# to kill previous instances and fix a bug to keep psmisc on persistent sessions
!sudo apt-get update
!sudo apt-get install psmisc
!fuser -k 6969/tcp


# Tunnels

# initialize without the share gradio command for other tunnels
share = ""

if Tunnel == "Gradio":
    share = "--share"
elif Tunnel == "Port Viewer":
    print("Be sure to run the manual steps in the guide above for Port Viewer!")
    import urllib
    endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip()
  
    # Construct the origin URL from the endpoint IP 
    ip_origin = f"http://{endpoint_ip}:{PORT}"

    clear_output()
    # hold the user-facing message
    public_url = "Please use the Port Viewer URL from the Lightning AI UI explained in the guide above."
    print(public_url)
elif Tunnel == "Ngrok":
    !pip install pyngrok --quiet
    from pyngrok import conf, ngrok

    MyConfig = conf.PyngrokConfig()
    MyConfig.auth_token = Token
    MyConfig.region = Region[0:2]
    conf.get_default().authtoken = Token
    conf.get_default().region = Region
    conf.set_default(MyConfig);

    ngrokConnection = ngrok.connect(PORT)
    public_url = ngrokConnection.public_url

    clear_output()
    print("Ngrok Public URL:", public_url)
elif Tunnel == "Cloudflare":
    # download cloudflare
    !curl -LO https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !sudo dpkg -i cloudflared-linux-amd64.deb
    # Run cloudflare
    import time
    import subprocess

    # clean up for previous instances
    !rm -rf nohup.out
    
    # Define the command as a list of arguments
    command = ["cloudflared", "tunnel", "--url", f"localhost:{PORT}"]

    # Open the output file
    with open("nohup.out", "w") as f:
        # Start the cloudflared process in the background using Popen
        # stdout and stderr are redirected to the file 'f'
        proc = subprocess.Popen(command, stdout=f, stderr=subprocess.STDOUT)

    time.sleep(7)
    # Find and print the Cloudflare URL with a prefix
    cloudflare_url = !grep -oE "https://[a-zA-Z0-9.-]+\.trycloudflare\.com" nohup.out

    clear_output()
    print("Cloudflare Public URL:", cloudflare_url[0])
elif Tunnel == "LocalTunnel":
    # install
    # Update package manager lists and install curl (used to download the script)
    !sudo apt-get update -y
    !sudo apt-get install -y curl

    # Download and run the NodeSource setup script for Node.js v18 (a stable LTS version).
    !curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash - 

    # Install the modern 'nodejs' package. This also includes npm.
    !sudo apt-get install -y nodejs
    print("Node.js & npm installation complete!")
        
    !sudo npm install -g localtunnel
    import time
    import urllib

    # clean up for previous instances
    !rm -rf url.txt
    
    # run localtunnel
    with open('url.txt', 'w') as file:
          file.write('')

    get_ipython().system_raw(f'lt --port {PORT} >> url.txt 2>&1 &')
    
    time.sleep(4)

    endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

    with open('url.txt', 'r') as file:
        public_url = file.read()
        public_url = public_url.replace("your url is: ", "")

    clear_output()
    print("LocalTunnel Public URL:", public_url)
    print("Local Tunnel Password:", endpoint_ip)
elif Tunnel == "Horizon":
    # install
    import os
    # Update package manager lists and install curl (used to download the script)
    !sudo apt-get update -y
    !sudo apt-get install -y curl

    # Download and run the NodeSource setup script for Node.js v18 (a stable LTS version).
    !curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash - 

    # Install the modern 'nodejs' package. This also includes npm.
    !sudo apt-get install -y nodejs
    print("Node.js & npm installation complete!")

    # clean up for previous instances
    !rm -rf url.txt
    
    # install horizon
    !sudo npm install -g @hrzn/cli
    !hrzn login $Token
    os.system(f"hrzn tunnel http://localhost:{PORT} >> url.txt 2>&1 &")
    time.sleep(5)

    with open('url.txt', 'r') as file:
      public_url = file.read()
      public_url = !grep -oE "https://[a-zA-Z0-9.-]+\.hrzn\.run" url.txt
      public_url = public_url[0]

    clear_output()
    print("Horizon Public URL:", public_url)
    

# Run Applio
%cd /teamspace/studios/this_studio/Applio
!python app.py --listen $share

# Delete everything

Lightning.ai sessions storage persists, so if there is any Applio updates, you need to delete everything first and then install it (with maybe a different branch variable).

In [ ]:
# program
!rm -rf /teamspace/studios/this_studio/Applio
# cloudflare tunnel temp
!rm -rf /teamspace/studios/this_studio/nohup.out
!rm -rf /teamspace/studios/this_studio/cloudflared-linux-amd64.deb
# localtunnel temp
!rm -rf /teamspace/studios/this_studio/url.txt

print("Sucessfully deleted everything!")